# Generate a metric

In [1]:
import pandas as pd
import numpy as np

# Generate a date range for 3 years of daily data
dates = pd.date_range(start='2021-01-01', end='2023-12-31', freq='D')
# Simulate some timeseries data
np.random.seed(42)
values = np.random.randn(len(dates)).cumsum()  # Example: cumulative sum of random values

# Create the timeseries DataFrame
ts_df = pd.DataFrame({'Date': dates, 'Value': values})

# Calculate a general metric: mean and standard deviation to date
mean_value = ts_df['Value'].mean()
std_value = ts_df['Value'].std()

print(f"Mean value to date: {mean_value:.2f}")
print(f"Standard deviation to date: {std_value:.2f}")

Mean value to date: 2.39
Standard deviation to date: 12.80


# Analyze and report

In [3]:
import plotly.express as px

# Add a 'Week' column to the DataFrame (ISO week)
ts_df['Week'] = ts_df['Date'].dt.isocalendar().week
ts_df['Year'] = ts_df['Date'].dt.isocalendar().year

# Combine year and week for unique week labels
ts_df['Year-Week'] = ts_df['Year'].astype(str) + '-W' + ts_df['Week'].astype(str).str.zfill(2)

# Create a box plot for each week's daily data
fig = px.box(ts_df, x='Year-Week', y='Value', points='all',
             title='Weekly Distribution of Daily Metric',
             labels={'Year-Week': 'Week', 'Value': 'Metric Value'})

# Optionally, show fewer x-ticks for readability
fig.update_xaxes(tickangle=45, tickmode='array', tickvals=ts_df['Year-Week'][::4])
# Reduce the number of x-tick labels for better readability
# Show only the first week of each quarter

# Find the first week of each quarter
quarter_starts = ts_df.groupby([ts_df['Year'], ((ts_df['Week'] - 1) // 13) + 1])['Year-Week'].first().values

fig.update_xaxes(
    tickmode='array',
    tickvals=quarter_starts,
    tickangle=45
)
fig.show()

# Dashboard

In [6]:
import dash
from dash import dcc, html, Input, Output

import plotly.express as px

# Predefined timeframes (as examples)
timeframes = {
    "2021 Q1": ("2021-01-01", "2021-03-31"),
    "2021 Q2": ("2021-04-01", "2021-06-30"),
    "2021 Q3": ("2021-07-01", "2021-09-30"),
    "2021 Q4": ("2021-10-01", "2021-12-31"),
    "2022 Q1": ("2022-01-01", "2022-03-31"),
    "2022 Q2": ("2022-04-01", "2022-06-30"),
    "2022 Q3": ("2022-07-01", "2022-09-30"),
    "2022 Q4": ("2022-10-01", "2022-12-31"),
    "2023 Q1": ("2023-01-01", "2023-03-31"),
    "2023 Q2": ("2023-04-01", "2023-06-30"),
    "2023 Q3": ("2023-07-01", "2023-09-30"),
    "2023 Q4": ("2023-10-01", "2023-12-31"),
}

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("Weekly Distribution of Daily Metric - Timeframe Comparison"),
    html.Div([
        html.Label("Select Timeframe 1:"),
        dcc.Dropdown(
            id='timeframe1',
            options=[{'label': k, 'value': k} for k in timeframes.keys()],
            value='2021 Q1'
        ),
        html.Label("Select Timeframe 2:"),
        dcc.Dropdown(
            id='timeframe2',
            options=[{'label': k, 'value': k} for k in timeframes.keys()],
            value='2023 Q1'
        ),
    ], style={'width': '40%', 'display': 'inline-block', 'verticalAlign': 'top'}),
    dcc.Graph(id='comparison-graph')
])

@app.callback(
    Output('comparison-graph', 'figure'),
    Input('timeframe1', 'value'),
    Input('timeframe2', 'value')
)
def update_graph(tf1, tf2):
    # Filter data for each timeframe
    tf1_start, tf1_end = timeframes[tf1]
    tf2_start, tf2_end = timeframes[tf2]
    df1 = ts_df[(ts_df['Date'] >= tf1_start) & (ts_df['Date'] <= tf1_end)].copy()
    df2 = ts_df[(ts_df['Date'] >= tf2_start) & (ts_df['Date'] <= tf2_end)].copy()
    df1['Timeframe'] = tf1
    df2['Timeframe'] = tf2
    compare_df = pd.concat([df1, df2])

    fig = px.box(
        compare_df,
        x='Year-Week',
        y='Value',
        color='Timeframe',
        points='all',
        title=f'Weekly Distribution: {tf1} vs {tf2}',
        labels={'Year-Week': 'Week', 'Value': 'Metric Value'}
    )

    # Show only the first week of each quarter for x-ticks
    quarter_starts = compare_df.groupby([compare_df['Year'], ((compare_df['Week'] - 1) // 13) + 1])['Year-Week'].first().values
    fig.update_xaxes(
        tickmode='array',
        tickvals=quarter_starts,
        tickangle=45
    )
    return fig

if __name__ == '__main__':
    # app.run(debug=True)
    app.run(host='localhost', port=8050, debug=True)